# Resource Maker

- creates resources using fhir.resources library
     1. load a skeleton yaml instance of a resource of type r_type
     2. create pymodel using costruct method
     3. update resource with elements defined elswhere like in a csv file based on:
         - element name
         - value
         
*step three is a work in progress*


### Set the current working directory

In [1]:
%cd '/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker'

/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker


### Import modules

In [2]:
from pathlib import Path
from fhir.resources import construct_fhir_element
from fhir.resources.fhirabstractmodel import ValidationError
from utils import new_dict, update_obj, validate, get_element_type, get_template
from datetime import datetime, date, timezone, timedelta

### Provide test data

get data from csv file instead of cell below

(need to start doing unit testing !!!)

In [3]:
print(f"===================================MY DATA:====================================\n {row}\n=============================================================================")

NameError: name 'row' is not defined

In [ ]:
extensions = {'instance_name': 'http://hl7.org/fhir/StructureDefinition/instance-name',
 'instance_description': 'http://hl7.org/fhir/StructureDefinition/instance-description'
             }

### Get skeleton resource to create resource model

this is a short cut to get started instead of building from the ground up.

In [ ]:
r_obj = get_template("Provenance")
print(r_obj.yaml())
print('-'*80)

### Add meta extensions

In [ ]:
extensions = {'instance_name': 'http://hl7.org/fhir/StructureDefinition/instance-name',
 'instance_description': 'http://hl7.org/fhir/StructureDefinition/instance-description'
             }

element = 'extension'
for k,v in extensions.items():
    value_x = 'valueString' if k == 'instance_name' else 'valueMarkdown'
    element_dict = new_dict(
                url = v,
                valueString = getattr(row,k) 
                )
    if k == 'instance_description' :
        element_dict['valueMarkdown'] = element_dict.pop('valueString') 
    update_obj(element,element_dict,r_obj)
    print(r_obj.yaml())
    print('-'*80)

### update the effective data

In [ ]:
r_obj.recorded = row.recorded
print(r_obj.yaml())
print('-'*80)

### Add the target

In [ ]:
element = 'target'
element_dict = new_dict(
            reference = row.target_fhirid,
            identifier = new_dict(value = row.target_identifier),  # business identifier
            display = row.target_display,
            )
update_obj(element,element_dict,r_obj,replace=True)
print(r_obj.yaml())
print('-'*80)

### Add the Author

In [ ]:
element = 'who'
element_dict = new_dict(
            reference = row.author_fhirid,
            identifier = new_dict(value = row.author_identifier, system = row.author_identifier_system ),  # business identifier
            display = row.author_display,
            )
print(element_dict)
update_obj(element,element_dict,r_obj.agent[0])
print(r_obj.yaml())
print('-'*80)

### Add the onBehalfOf

In [ ]:
element = 'onBehalfOf'
try:
    element_dict = new_dict(
                reference = row.onbehalfof_fhirid,
                identifier = new_dict(value = row.onbehalfof_identifier, system = row.onbehalfof_identifier_system),  # business identifier
                display = row.onbehalfof_display,
                )
except AttributeError as e:
    print(e)
else:
    update_obj(element,element_dict,r_obj.agent[0])
    print(r_obj.yaml())
    print('-'*80)

### Add the gateway as transmitter

use transmitter for now and consider other types later

In [ ]:
element = 'agent'
agent_type_system = "http://hl7.org/fhir/us/core/CodeSystem/us-core-provenance-participant-type"
agent_type_code = "transmitter"
agent_type_display = "Transmitter"

who_dict = new_dict(
            reference = row.transmitter_fhirid,
            identifier = new_dict(value = row.transmitter_identifier),  # business identifier
            display = row.transmitter_display,
            )

coding_dict = new_dict(
            system = agent_type_system,
            code = agent_type_code,
            display = agent_type_display,
            )

element_dict =  new_dict(
      type = new_dict(coding = [coding_dict],
             text='Transmitter'),
      who = who_dict,
    )

try:
    update_obj(element,element_dict,r_obj)
except ValidationError as e:
    print(e)
print('-'*80)
print(r_obj.yaml())
print('-'*80)

### Validate

In [ ]:
validate(r_obj)

### Save Locally

In [ ]:
ig_source_path = f'examples/{row.r_type}'
path = Path.cwd() / ig_source_path / f'example.yaml' 
print(f'...........saving to file {path}............')
path.write_text(r_obj.yaml())

Testing implicitly building resource based upon the labels:
    
    maybe use a template with nones and scrub the null values.